# Getting Started With the SHFSG+

## Device Setup and Data Server

Let's start by creating a `DeviceSetup` with a single SHFSG+ instrument. To learn more about what a `DeviceSetup` is and how to use it, have a look at [this page](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/00_device_setup/concepts/index.html).

First, we import `laboneq.simple`, which contains the `DeviceSetup`.

In [1]:
from laboneq.simple import *

Create the `DeviceSetup` with the information about the data server it should connect to. Here, we use "localhost". Change this to the LabOne data server that is running on your computer. You can find the IP address of LabOne data server by navigating to Config > Data Server > Host in the LabOne UI.

Note that calling `add_dataserver` below does not attempt to make the connection yet. This will be done only upon calling `Session.connect()`.

In [2]:
device_setup = DeviceSetup("ZI_SHFSG+")
device_setup.add_dataserver(
    host="111.22.33.44",
    port="8004",
)

Create an `SHFSG` instrument, which was imported from `laboneq.simple`, and add it to the `DeviceSetup`. 

When creating the instrument instance, you need to specify the device ID under `address`; for example, `dev12345`.

If you do not have an active LabOne data server running to connect to the instrument, you also need to specify the `device_options` that are installed on your instrument. These options are used to ensure a correct experiment compilation for your system.

The possible options you can set for an SHFSG instrument are:
* either `"SHFSG8"` if you have an 8-channel instrument or `"SHFSG4"` for the 4-channel version;
* `PLUS` if you have the SHFSG+;
* `"RTR"` for the output router and adder option.

When passing these options to the instrument, add them in any order, separated by a forward slash (`"/"`). Below, we will use `device_options="SHFSG8/PLUS/RTR"`. Have a look at [this page](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/00_device_setup/concepts/01_instrument_options.html) to learn more about the possible options of our instruments.

You can also pass additional input parameters to configure your instrument:
  
* the `interface` over which to connect to the instrument, either `"1GbE"` (default) or `"usb"`. Note that **to ensure the stability of the connection to the instrument, we recommend to use the ethernet interface instead of USB**;

* the `reference_clock_source` for the instrument, either as `"internal"` to use the instrument's own internal reference clock, or `"external"` (default) if you are using an external source like the PQSC instrument, for example.

In [3]:
shfsg = SHFSG(
    uid="shfsg",
    address="dev12345",
    interface="1GbE",
    device_options="SHFSG8/PLUS/RTR",
    reference_clock_source="internal",
)

device_setup.add_instruments(shfsg)

Next, we create connections to each of the 8 ports of the instrument and add these connections to the `DeviceSetup`. These connections are represented in LabOne Q as [logical signal lines](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/02_logical_signals/concepts/index.html) between the instruments and your device under test. Here, we assume the device under test is an eight-qubit QPU and use the physical lines of these qubits as a naming convention for our signal lines. Note that the instances of the `LogicalSignal`s will be created automatically by the `add_connections` method of `DeviceSetup`, with the names that we have specified.

In [4]:
# Create drive lines for all 8 qubits, connected to the 8 SG channels on the instrument.

for idx in range(8):
    device_setup.add_connections(
        "shfsg",
        create_connection(
            to_signal=f"q{idx}/drive", ports=f"SGCHANNELS/{idx}/OUTPUT", type="iq"
        ),
    )

You can inspect the `LogicalSignal`s that have been created by calling `device_setup.logical_signal_by_uid(signal_name)`; for example:

In [5]:
device_setup.logical_signal_by_uid("q0/drive")

LogicalSignal(
│   uid='q0/drive',
│   direction=IODirection.OUT,
│   name='drive',
│   calibration=None,
│   path='/logical_signal_groups/q0/drive',
│   physical_channel=PhysicalChannel(
│   │   uid='shfsg/sgchannels_0_output',
│   │   name='sgchannels_0_output',
│   │   type=<PhysicalChannelType.IQ_CHANNEL: 'iq_channel'>,
│   │   path='/physical_channel_groups/shfsg/sgchannels_0_output',
│   │   calibration=None
│   )
)


Next, we need to configure the `calibration` of the signal lines of the `DeviceSetup`. We will set the most common properties:

* `range` - the output power range in dBm;
* `local_oscillator` - an instance of `Oscillator` where we specify the local oscillator frequency;
* `oscillator` - an instance of `Oscillator` where we specify the IF frequency that will be mixed with local oscillator frequency.

Keep in mind that the channels in each of the channel pairs 0-1, 2-3, 4-5, 6-7 share the local oscillator, so the local oscillator for these channels in these channel pairs must be configured with the same frequency value.

In [9]:
drive_lo_01 = Oscillator(frequency=6e9)
drive_lo_23 = Oscillator(frequency=6.2e9)
drive_lo_45 = Oscillator(frequency=6.4e9)
drive_lo_67 = Oscillator(frequency=6.6e9)

config = Calibration()
for idx, drive_lo in enumerate([drive_lo_01, drive_lo_23, drive_lo_45, drive_lo_67]):
    config[f"q{2 * idx}/drive"] = SignalCalibration(
        local_oscillator=drive_lo, oscillator=Oscillator(frequency=100e6), range=10
    )
    config[f"q{2 * idx + 1}/drive"] = SignalCalibration(
        local_oscillator=drive_lo, oscillator=Oscillator(frequency=100e6), range=10
    )

# Apply the configuration to the DeviceSetup
device_setup.set_calibration(config)

Great! We have our `DeviceSetup` for a single SHFSG+ instrument.

Before we can play a signal on the instrument, we first have to connect it to the LabOne data server via the LabOne Q [Session](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/01_session/concepts/index.html). Here, we connect in emulation mode by calling `Session.connect` with `do_emulation=True`. Set this flag to False in order to connect to a physical setup.

In [ ]:
session = Session(device_setup)
session.connect(do_emulation=True)  # do_emulation=False when at a physical setup

## Generate a Simple Experiment

Let's create a simple experiment that plays back a 200-ns gaussian pulse on all of the 8 drive channels. In addition, we will sweep the amplitude of the pulses.

To learn more about the `Experiment` object and how to write experiments in LabOne Q, have a look at the ["Experiment Definition"](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/05_experiment/concepts/index.html) and ["Writing and Experiment Workflow"](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/writing_experiments.html#write-the-experiment-pulse-sequence) sections of the manual.

In [ ]:
import numpy as np


@dsl.experiment(signals=[f"q{idx}_drive" for idx in range(8)])
def simple_experiment(count):
    with dsl.acquire_loop_rt(
        count=count,
        averaging_mode=AveragingMode.CYCLIC,
        acquisition_type=AcquisitionType.INTEGRATION,
    ):
        with dsl.sweep(
            name="amplitude_sweep",
            parameter=SweepParameter("drive_pulse_amplitudes", np.linspace(0, 1, 7)),
        ) as amplitude:
            for idx in range(8):
                with dsl.section(name=f"play-drive-pulse_q{idx}"):
                    dsl.play(
                        f"q{idx}_drive",
                        pulse_library.gaussian(amplitude=1, length=200e-9),
                        amplitude=amplitude,
                    )
                    dsl.delay(f"q{idx}_drive", time=0.5e-6)

    # Map the ExperimentSignals "q{idx}_drive" to the logical signal lines defined in the `DeviceSetup`
    for idx in range(8):
        dsl.map_signal(f"q{idx}_drive", f"q{idx}/drive")

Next, we instantiate the `Experiment` by running the function `simple_experiment`.

In [ ]:
experiment = simple_experiment(5)

Note that, the mapping between the `ExperimentSignal`s and the logical signal lines do not have to be done as part of `simple_experiment` but can also be done or modified on the `Experiment` instance returned by `simple_experiment` as follows:

```python
q0_ls = device_setup.logical_signal_groups["q0"].logical_signals
experiment.map_signal(f"q{idx}_drive", q_ls["drive"])
```

You can also map an `ExperimentSignal` directly to a `LogicalSignal` instead of its UID, as written above.

Compile the simple_experiment and inspect it using `plot_simulation`:

In [ ]:
compiled_experiment = session.compile(experiment)

In [ ]:
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation

plot_simulation(compiled_experiment, start_time=0, length=5e-6)

You can inspect your pulse sequence in more detail by using the interactive pulse-sheet viewer. Calling the function `show_pulse_sheet` creates an HTML file that you can open in your browser. To show the pulse sheet in the kernel, set `interactive=True`.

In [ ]:
show_pulse_sheet("simple_pulse_sequence", compiled_experiment, interactive=True)

Finally, let's run the pulse sequence on the instrument.

In [ ]:
_ = session.run()